In [42]:
#!/.conda/envs/learn python
# -*- coding: utf-8 -*-

"""
模型可视化
230226该程序目前用不了
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
wanghaoyu191@mails.ucas.ac.cn
"""

'\n模型可视化\n~~~~~~~~~~~~~~~~\ncode by wHy\nAerospace Information Research Institute, Chinese Academy of Sciences\n751984964@qq.com\n'

In [43]:
#coding: utf-8
import os
from PIL import Image
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys

sys.path.append(r"D:\github_repository\AGRS_semantic_segmentation")

from networks.DLinknet import DLinkNet34
from networks.Unet import Unet

In [44]:
def draw_CAM(model_my, img_path, save_path, transform=None, visual_heatmap=False):
    '''
    绘制 Class Activation Map
    :param model: 加载好权重的Pytorch model
    :param img_path: 测试图片路径
    :param save_path: CAM结果保存路径
    :param transform: 输入图像预处理方法
    :param visual_heatmap: 是否可视化原始heatmap（调用matplotlib）
    :return:
    '''
    # 图像加载&预处理
    img = Image.open(img_path).convert('RGB')
    if transform:
        img = transform(img)
    # img = img.unsqueeze(0)
 
    # 获取模型输出的feature/score
    features = model_my.features(img)
    output = model_my.classifier(features)
 
    # 为了能读取到中间梯度定义的辅助函数
    def extract(g):
        global features_grad
        features_grad = g
 
    # 预测得分最高的那一类对应的输出score
    pred = torch.argmax(output).item()
    pred_class = output[:, pred]
 
    features.register_hook(extract)
    pred_class.backward() # 计算梯度
 
    grads = features_grad   # 获取梯度
 
    pooled_grads = torch.nn.functional.adaptive_avg_pool2d(grads, (1, 1))
 
    # 此处batch size默认为1，所以去掉了第0维（batch size维）
    pooled_grads = pooled_grads[0]
    features = features[0]
    # 512是最后一层feature的通道数
    for i in range(512):
        features[i, ...] *= pooled_grads[i, ...]
 
    # 以下部分同Keras版实现
    heatmap = features.detach().numpy()
    heatmap = np.mean(heatmap, axis=0)
 
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
 
    # 可视化原始热力图
    if visual_heatmap:
        plt.matshow(heatmap)
        plt.show()
 
    img = cv2.imread(img_path)  # 用cv2加载原始图像
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))  # 将热力图的大小调整为与原始图像相同
    heatmap = np.uint8(255 * heatmap)  # 将热力图转换为RGB格式
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # 将热力图应用于原始图像
    superimposed_img = heatmap * 0.4 + img  # 这里的0.4是热力图强度因子
    cv2.imwrite(save_path, superimposed_img)  # 将图像保存到硬盘


In [48]:
model_path = r'E:\project_UAV\3-weights\UNet-UAV_building_1008.th'
num_class = 2
band_num = 3
img_path = r'E:\project_UAV\1-clip_img\_1.tif'
save_path = r'D:\1data\230223_vis_test\vis_test_230223_test_1.png'

net = Unet(num_classes = num_class, band_num = band_num)
net = net.cuda()
net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
net.load_state_dict(torch.load(model_path))
if isinstance(net, torch.nn.DataParallel):
    net = net.module
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)
net.eval()

draw_CAM(model_my=net, img_path = img_path, save_path = save_path, visual_heatmap = True)

ModuleAttributeError: 'Unet' object has no attribute 'features'